# Udemy Apache Beam tutorial 
# SET UP

In [ ]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')
run('pip install beam-nuggets')
# Copy the input file into the local file system.
run('mkdir -p data')
run('mkdir -p outputs')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')
import apache_beam as beam
import re

>> pip install --quiet apache-beam
     |████████████████████████████████| 8.6MB 2.7MB/s 
     |████████████████████████████████| 63.8MB 65kB/s 
     |████████████████████████████████| 1.4MB 42.8MB/s 
     |████████████████████████████████| 829kB 48.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 153kB 52.9MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 112kB 50.6MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have fo

In [ ]:
from google.colab import files
upload = files.upload()

Saving exclude_ids.txt to exclude_ids.txt


In [ ]:
run('mv ./*.txt ./data/')

>> mv ./*.txt ./data/



# Colab DEMO

In [ ]:
run('head -n 20 ./data/kinglear.txt')

In [ ]:
import apache_beam as beam
import re

inputs_pattern = 'data/kinglear.txt'
outputs_prefix = 'outputs/part'

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      # | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

# Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 {}-00000-of-*'.format(outputs_prefix))

>> head -n 20 outputs/part-00000-of-*
('KING', 243)
('LEAR', 236)
('DRAMATIS', 1)
('PERSONAE', 1)
('king', 65)
('of', 447)
('Britain', 2)
('OF', 15)
('FRANCE', 10)
('DUKE', 3)
('BURGUNDY', 8)
('CORNWALL', 63)
('ALBANY', 67)
('EARL', 2)
('KENT', 156)
('GLOUCESTER', 141)
('EDGAR', 126)
('son', 29)
('to', 438)
('Gloucester', 26)



In [ ]:
re.findall(r"[a-zA-Z0-9']+", 'dctjdc 4243 asfa')

['dctjdc', '4243', 'asfa']

# Branching Pipelines

In [ ]:
run('cp ./data/dept_data.txt ./data/dept_data1.txt')
run('head -n 20 ./data/dept_data.txt')

>> cp ./data/dept_data.txt ./data/dept_data1.txt

>> head -n 20 ./data/dept_data.txt
149633CM,Marco,10,Accounts,1-01-2019
212539MU,Rebekah,10,Accounts,1-01-2019
231555ZZ,Itoe,10,Accounts,1-01-2019
503996WI,Edouard,10,Accounts,1-01-2019
704275DC,Kyle,10,Accounts,1-01-2019
957149WC,Kyle,10,Accounts,1-01-2019
241316NX,Kumiko,10,Accounts,1-01-2019
796656IE,Gaston,10,Accounts,1-01-2019
331593PS,Beryl,20,HR,1-01-2019
560447WH,Olga,20,HR,1-01-2019
222997TJ,Leslie,20,HR,1-01-2019
171752SY,Mindy,20,HR,1-01-2019
153636AS,Vicky,20,HR,1-01-2019
745411HT,Richard,20,HR,1-01-2019
298464HN,Kirk,20,HR,1-01-2019
783950BW,Kaori,20,HR,1-01-2019
892691AR,Beryl,20,HR,1-01-2019
245668UZ,Oscar,20,HR,1-01-2019
231206QD,Kumiko,30,Finance,1-01-2019
357919KT,Wendy,30,Finance,1-01-2019



In [ ]:
inputs_pattern = './data/dept_data*'
outputs_prefix = './outputs/branching-dept1'

with beam.Pipeline() as p:
  input_collection = (
      p
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'split lines' >> beam.Map(lambda lines: lines.split(','))
  )

  accounts_count = (
      input_collection
      | 'find Accounts' >> beam.Filter(lambda record: record[3]=='Accounts')
      | 'Pair each accounts emplyee with 1' >> beam.Map(lambda record: ('Accounts: '+record[1], 1))
      | 'Group and sum accounts' >> beam.CombinePerKey(sum)
  )
      
  hr_count = (
      input_collection
      | 'find HR' >> beam.Filter(lambda record: record[3]=='HR')
      | 'Pair each hr emplyee with 1' >> beam.Map(lambda record: ('HR: '+record[1], 1))
      | 'Group and sum hr' >> beam.CombinePerKey(sum)
  )
      
  output_collection = (
      (accounts_count, hr_count)
      | 'combine 2 result' >> beam.Flatten()
      | 'output files' >> beam.io.WriteToText(outputs_prefix)
  )


# Pardo Transform

In [ ]:
import apache_beam as beam

inputs_pattern = './data/dept_data*'
outputs_prefix = './outputs/dept1'


class SplitRow(beam.DoFn):

  def process(self, element):
    return [element.split(',')]


class FilterAccounts(beam.DoFn):

  def process(self, element):
    if element[3]=='Accounts':
      return [element]


class PairEmployees(beam.DoFn):
  
  def process(self, element):
    return [(element[3]+','+element[1], 1)]


class Counting(beam.DoFn):

  def process(self, element):
    (key, values) = element    # ['Marco, Accounts', [1,1,1,1,1,1,1]]
    return [(key, sum(values))]


with beam.Pipeline() as p:
  input_collection = (
    p
    | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
    | 'Split lines' >> beam.ParDo(SplitRow())
    | 'Filter Accounts' >> beam.ParDo(FilterAccounts())
    | 'Mapping' >> beam.ParDo(PairEmployees())
    | 'Group by key:1' >> beam.GroupByKey() # ['Marco, Accounts', [1,1,1,1,1,1,1]]
    | 'Sum using ParDo' >> beam.ParDo(Counting())
    | 'output file' >> beam.io.WriteToText(outputs_prefix)
  )

# Combiner

In [ ]:
inputs_pattern = './data/dept_data*'
outputs_prefix = './outputs/combiner_dept'

class AverageFn(beam.CombineFn):

  def create_accumulator(self):
    return (0.0, 0)   # initialize sum_count

  def add_input(self, sum_count, input):  # add all elements (sum, count)
    print(sum_count, input)
    (sum, count) = sum_count
    return sum+input, count+1

  def merge_accumulators(self, accumulators):
    print(accumulators)
    ind_sums, ind_counts = zip(*accumulators)   # zip - [(27,3), (39,3), (18,2)] -->  [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)    # (84,8)

  def extract_output(self, sum_count):
    (sum, count) = sum_count    # combine globally using CombineFn
    return (sum / count) if count else float('NaN') 


with beam.Pipeline() as p:
  small_sum = (
    p
    | beam.Create([])
    | beam.CombineGlobally(AverageFn())
    | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

# Composite Transforms

In [ ]:
inputs_pattern = './data/dept_data*'
outputs_prefix = './outputs/Composite-Transforms-dept1'


def SplitRow(element):
  return element.split(',')

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

def format_output(element):
  name, count = element
  return ' ,'.join((name, str(count), 'Regular employee'))


class MyTransForm(beam.PTransform):

  def expand(self, input_coll):
    a = (
        input_coll
        |'Group and sum key:1' >> beam.CombinePerKey(sum)
        |'Count filter' >> beam.Filter(filter_on_count)
        |'Regular employees' >> beam.Map(format_output)
    )
    return a



with beam.Pipeline() as p:
  input_collection = (
      p
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'split lines' >> beam.Map(lambda lines: lines.split(','))
  )

  accounts_count = (
      input_collection
      | 'find Accounts' >> beam.Filter(lambda record: record[3]=='Accounts')
      | 'Pair each accounts emplyee with 1' >> beam.Map(lambda record: ('Accounts '+record[1], 1))
      | 'Composite accounts' >> MyTransForm()
  )
      
  hr_count = (
      input_collection
      | 'find HR' >> beam.Filter(lambda record: record[3]=='HR')
      | 'Pair each hr emplyee with 1' >> beam.Map(lambda record: ('HR '+record[1], 1))
      | 'Composite hr' >> MyTransForm()
  )
      
  output_collection = (
      (accounts_count, hr_count)
      | 'combine 2 result' >> beam.Flatten()
      | 'output files' >> beam.io.WriteToText(outputs_prefix)
  )

run('head -n 20 ./outputs/Composite-Transforms-dept1*')

>> head -n 20 ./outputs/Composite-Transforms-dept1*
Accounts Marco ,31 ,Regular employee
Accounts Rebekah ,31 ,Regular employee
Accounts Itoe ,31 ,Regular employee
Accounts Edouard ,31 ,Regular employee
Accounts Kyle ,62 ,Regular employee
Accounts Kumiko ,31 ,Regular employee
Accounts Gaston ,31 ,Regular employee
HR Beryl ,62 ,Regular employee
HR Olga ,31 ,Regular employee
HR Leslie ,31 ,Regular employee
HR Mindy ,31 ,Regular employee
HR Vicky ,31 ,Regular employee
HR Richard ,31 ,Regular employee
HR Kirk ,31 ,Regular employee
HR Kaori ,31 ,Regular employee
HR Oscar ,31 ,Regular employee



# CoGroupByKey

In [ ]:
outputs_prefix = './outputs/GoGroupByKey-dept1'


def retTuple(element):
  thisTuple = element.split(',')
  return (thisTuple[0], thisTuple[1:])


with beam.Pipeline() as p:
  dept_rows = (
      p
      | 'Read dept lines' >> beam.io.ReadFromText('./data/dept_data*')
      | 'Pair each employee with key' >> beam.Map(retTuple)
  )

  loc_rows = (
      p
      | 'Read location lines' >> beam.io.ReadFromText('./data/location*')
      | 'Pair each location with key' >> beam.Map(retTuple)
  )
      
  results = (
      {'dept_data': dept_rows, 'loc_data': loc_rows}
      | 'Group by key' >> beam.CoGroupByKey()
      | 'output files' >> beam.io.WriteToText(outputs_prefix)
  )

run('head -n 20 ./outputs/GoGroupByKey-dept1*')

>> head -n 20 ./outputs/GoGroupByKey-dept1*
('149633CM', {'dept_data': [['Marco', '10', 'Accounts', '1-01-2019'], ['Marco', '10', 'Accounts', '2-01-2019'], ['Marco', '10', 'Accounts', '3-01-2019'], ['Marco', '10', 'Accounts', '4-01-2019'], ['Marco', '10', 'Accounts', '5-01-2019'], ['Marco', '10', 'Accounts', '6-01-2019'], ['Marco', '10', 'Accounts', '7-01-2019'], ['Marco', '10', 'Accounts', '8-01-2019'], ['Marco', '10', 'Accounts', '9-01-2019'], ['Marco', '10', 'Accounts', '10-01-2019'], ['Marco', '10', 'Accounts', '11-01-2019'], ['Marco', '10', 'Accounts', '12-01-2019'], ['Marco', '10', 'Accounts', '13-01-2019'], ['Marco', '10', 'Accounts', '14-01-2019'], ['Marco', '10', 'Accounts', '15-01-2019'], ['Marco', '10', 'Accounts', '16-01-2019'], ['Marco', '10', 'Accounts', '17-01-2019'], ['Marco', '10', 'Accounts', '18-01-2019'], ['Marco', '10', 'Accounts', '19-01-2019'], ['Marco', '10', 'Accounts', '20-01-2019'], ['Marco', '10', 'Accounts', '21-01-2019'], ['Marco', '10', 'Accounts', '22-01

# Access Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Side Inputs

In [ ]:
side_list = list()

with open('./data/exclude_ids.txt', 'r') as my_file:
  for line in my_file:
    side_list.append(line.rstrip())

class FilterUsingLength(beam.DoFn):
  def process(self, element, side_list, lower_bound, upper_bound=float('inf')):
    element_list = element.split(',')
    id = element_list[0]
    name = element_list[1]
    id = id
    if ((lower_bound<=len(name)<=upper_bound) and (id not in side_list)):
      return [element_list]


def Format_output( element):
  print(element)
  return ', '.join(map(lambda x: str(x), element))

# def format_output(element):
#   name, count = element
#   return ' ,'.join((name, str(count), 'Regular employee'))


with beam.Pipeline() as pl:
  small_names = (
      pl
      | 'Read from text file' >> beam.io.ReadFromText('./data/dept*')
      | 'ParDo with side_list' >> beam.ParDo(FilterUsingLength(), side_list, 3, 10)
      | beam.Filter(lambda record: record[3]=='Accounts')
      | beam.Map(lambda record: (record[0]+" "+record[1], 1))
      | beam.CombinePerKey(sum)
      | 'Format results' >> beam.Map(Format_output)
      | beam.io.WriteToText('./outputs/Side_Inputs')
  )

('503996WI Edouard', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)


# Additional Outputs in Pipeline

## 註解的部分為 條件若衝突則無法取得資訊
## exp: long_name short_name 將所有的可能都return 所以根本找不到開頭為K的人名
# 與branch觀念做合併即可解決

In [ ]:
class ProcessWords(beam.DoFn):

  def process(self, element, cutoff_lenth, marker):
    name = element.split(',')[1]

    if len(name) <= cutoff_lenth:
      return [beam.pvalue.TaggedOutput('Short_Names', name)]
    else:
      return [beam.pvalue.TaggedOutput('Long_Names', name)]



class ProcessWordsFirst(beam.DoFn):

  def process(self, element, marker):
    name = element.split(',')[1]
    if name.startswith(marker):
      return [name]


with beam.Pipeline() as p:

  read_file = (
      p
      | beam.io.ReadFromText('./data/dept_data*')
  )

  results = (
      read_file
      | beam.ParDo(ProcessWords(), cutoff_lenth=4, marker='K').with_outputs('Short_Names', 'Long_Names', main='Names_K')
  )

  Names_K = (
      read_file
      | beam.ParDo(ProcessWordsFirst(), marker='K')
  )

  short_collection = results.Short_Names
  long_collection = results.Long_Names
  # startK_collection = results.Names_K

  short_collection | 'Write short' >> beam.io.WriteToText('outputs/short')
  long_collection | 'Write long' >> beam.io.WriteToText('outputs/long')
  test = (
      Names_K
      | 'Write K name' >> beam.io.WriteToText('outputs/start_k')
  )
  # Names_K | 'Write K name' >> beam.io.WriteToText('outputs/start_k')

In [ ]:
'Jason'.startswith('J')

True

# neggets

In [ ]:
import argparse
import json
import logging
import time

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam.transforms.window as window


# from apache_beam.runners import DataflowRunner

In [ ]:
# Defines the BigQuery schema for the output table.
SCHEMA = ','.join([
    'msg:STRING'
])


def parse_json_message(message):
    """Parse the input json message and add 'score' & 'processing_time' keys."""
    row = json.loads(message)
    return {
        'msg': row['msg']
    }


# def get_statistics(url_messages):
#     """Get statistics from the input URL messages."""
#     url, messages = url_messages
#     return {
#         'url': url,
#         'num_reviews': len(messages),
#         'score': sum(msg['score'] for msg in messages) / len(messages),
#         'first_date': min(msg['processing_time'] for msg in messages),
#         'last_date': max(msg['processing_time'] for msg in messages),
#     }


def run(args, input_subscription, output_table, window_interval):
    """Build and run the pipeline."""
    # options = PipelineOptions(args, save_main_session=True, streaming=True, runner='DataflowRunner',
    #                           project='tacc-269308', job_name='deploy-test', temp_location='gs://dataproc-7301a3f4-8f83-45e9-ad78-fbeef9446c45-asia-east1/dataflow/temp', 
    #                           region='asia-east1-b')
    options = PipelineOptions(args, save_main_session=True, streaming=True, runner='DataflowRunner',
                              project='jw001-practice', job_name='deploy-test', temp_location='gs://dataflow_staging_tmp/temp', 
                              region='asia-east1-b')

    # options = PipelineOptions(args, save_main_session=True, streaming=True)


    with beam.Pipeline(options=options) as pipeline:

        # Read the messages from PubSub and process them.
        messages = (
            pipeline
            | 'Read from Pub/Sub' >> beam.io.ReadFromPubSub(
                subscription=input_subscription).with_output_types(bytes)
            | 'UTF-8 bytes to string' >> beam.Map(lambda msg: msg.decode('utf-8'))
            | 'Parse JSON messages' >> beam.Map(parse_json_message))

        # Output the results into BigQuery table.
        _ = messages | 'Write to Big Query' >> beam.io.WriteToBigQuery(
            output_table, schema=SCHEMA)

In [ ]:
if __name__ == '__main__':
#     logging.getLogger().setLevel(logging.INFO)
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--output_table',
        default='tacc-269308:dataflow2bigquery.sub_msg', 
        help='Output BigQuery table for results specified as: '
        'PROJECT:DATASET.TABLE or DATASET.TABLE.')
    parser.add_argument(
        '--input_subscription',
        default='projects/tacc-269308/subscriptions/sub2bigquery', 
        help='Input PubSub subscription of the form '
        '"projects/<PROJECT>/subscriptions/<SUBSCRIPTION>."')
    parser.add_argument(
        '--window_interval',
        default=60,
        help='Window interval in seconds for grouping incoming messages.')
    
    known_args, pipeline_args = parser.parse_known_args()
    
    run(pipeline_args, known_args.input_subscription, known_args.output_table,
        known_args.window_interval)